# Consolidated application data

In [1]:
import re as re

import numpy as np
import pandas as pd

pd.options.display.max_columns = 50
pd.options.display.max_rows = 250
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:.2f}".format

from itertools import chain
from collections import Counter
from itertools import combinations

from calitp import *
from siuba import *

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/consolidated_applications/"

In [2]:
data = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}Copy of Application_Review_Report_5_2_2022.xls")
)

In [3]:
data.shape

(346, 24)

In [4]:
data.isna().sum()

organization_name                            0
district                                     6
application_name                             0
year                                         0
application_status                           0
project_upin                                 0
project_category                             0
project_line_item__ali_                      0
project_description                          0
is_stimulus                                  0
consolidated_application                     0
total_expenses                               0
_5311_funds                                  7
_5311_f__funds                               7
_5311_cmaq_funds                             7
_5339_funds                                  7
federal_total                                0
other_fed_funds_total                        7
lctop__state__funds                          7
sb1__state_of_good_repair__state__funds      7
transit_development_act__state__funds        7
other_state_f

In [5]:
data.dtypes

organization_name                           object
district                                   float64
application_name                            object
year                                         int64
application_status                          object
project_upin                                object
project_category                            object
project_line_item__ali_                     object
project_description                         object
is_stimulus                                 object
consolidated_application                    object
total_expenses                               int64
_5311_funds                                float64
_5311_f__funds                             float64
_5311_cmaq_funds                           float64
_5339_funds                                float64
federal_total                                int64
other_fed_funds_total                      float64
lctop__state__funds                        float64
sb1__state_of_good_repair__stat

In [6]:
f"{data.organization_name.nunique()} unique organizations"

'121 unique organizations'

In [7]:
f"{data.project_upin.nunique()} different projects" 

'346 different projects'

In [8]:
data.consolidated_application.value_counts()

Yes    343
No       3
Name: consolidated_application, dtype: int64

In [9]:
data.application_status.value_counts()

Submitted        338
Not Submitted      8
Name: application_status, dtype: int64

In [10]:
data.head(1)

,organization_name,district,application_name,year,application_status,project_upin,project_category,project_line_item__ali_,project_description,is_stimulus,consolidated_application,total_expenses,_5311_funds,_5311_f__funds,_5311_cmaq_funds,_5339_funds,federal_total,other_fed_funds_total,lctop__state__funds,sb1__state_of_good_repair__state__funds,transit_development_act__state__funds,other_state_funds,state_total,local_total
0,Alameda-Contra Costa Transit District (AC Transit),4.00,Consolidated Application (Consolidated Application),2022,Submitted,BCG0003649,OP,300901,Operating Assistance (LCTOP Only - Project # 1),No,Yes,27795000,0.00,0.00,0.00,0.00,0,0.00,6546867.00,0.00,0.00,0.00,6546867.00,"District operating funds: $21,248,133.00\nTotal Local: $21,248,133.00"


## Organizations
* 121 unique orgs

In [11]:
# Remove any acronyms
data["organization_name"] = data["organization_name"].str.replace(
    "\s+\(.*$", "", regex=True
)

In [12]:
# Replace Ventura, read in weirdly
data["organization_name"] = data["organization_name"].replace(
    {
        "Ventura County Transportation Commission\xa0": "Ventura County Transportation Commission"
    }
)

## Project Columns

In [13]:
## Change acronyms to full name. What does PL and CM mean?
data.project_category.unique()

array(['OP', 'CA', 'PL', 'CM'], dtype=object)

In [14]:
data["project_category"] = data["project_category"].replace(
    {
        "OP": "Operating",
        "CA": "Capital",
    }
)

### Project Descriptions
The descriptions are long and there are 200+ of them. The project category included in the data set is pretty vague, manipulate for something in between.

In [15]:
data["project_description"] = data["project_description"].str.lower()

In [16]:
data.project_description.nunique()

206

In [17]:
data.project_description.unique()

array(['operating assistance (lctop only - project # 1)',
       'purchase misc communications equip (lctop only - project # 2)',
       'operating assistance sliding scale',
       'purchase replacement < 30 ft bus',
       'lctop - purchase replacement van',
       'operating assistance sliding scale - ffy2022',
       'operating assistance sliding scale - ffy2023',
       'operating assistance - free fares',
       'operating assistance sliding scale fy 22',
       'operating assistance sliding scale fy 23',
       'buy 35-39 ft bus for expansion',
       'support equip/facilities-equipment (microgrid etc)',
       'operating assistance', 'operating assistance - (lctop only) #1',
       'operating assistance (lctop only)',
       'operating assistance sliding scale ffy2022',
       'operating assistance sliding scale ffy2023',
       'purchase replacement std 30-34\xa0ft bus (lctop only)',
       'construction - bus shelters (lctop only)',
       'operating assistance sliding scale 

In [18]:
data["short_description"] = data["project_description"].str.extract(
    "(operating|bus|construction|buses|planning|van|vessel|fares|ridership|vehicle|station|service|equipment|maintenance|surveillance|renovate|free|equip|operational)",
    expand=False,
)

In [19]:
data["short_description"] = data["short_description"].replace(
    {
        "operating": "operating assistance",
        "operational": "operating assistance",
        "free": "free fare program",
        "ridership": "ridership expansion",
        "fare": "free fare programs",
        "service": "service expansion",
        "buses": "purchasing vehicles",
        "bus": "purchasing vehicles",
        "van": "purchasing vehicles",
        "vessel": "purchasing vehicles",
        "vehicles": "purchasing vehicles",
        "vehicle": "purchasing vehicles",
        "planning": "transit planning",
        "station": "construction",
        "construction": "construction",
        "maintenance": "maintenance/renovation",
        "renovate": "maintenance/renovation",
        "equipment": "purchasing other tech",
        "equip": "purchasing other tech",
        "surveillance": "purchasing other tech",
    }
)

In [20]:
data["short_description"] = data["short_description"].fillna("other category")

In [21]:
data["short_description"]= data["short_description"].str.title()

In [22]:
data.short_description.value_counts()

Operating Assistance      204
Purchasing Vehicles        89
Construction               16
Maintenance/Renovation      8
Purchasing Other Tech       7
Free Fare Program           7
Other Category              6
Service Expansion           5
Transit Planning            3
Ridership Expansion         1
Name: short_description, dtype: int64

In [23]:
#data[['project_description','short_description']].tail(250)

## Monetary Columns

<b> Local Total </b> 
* This column represents the different types of local funding a project can receive.
* Extract everything after the colons. 

In [24]:
data["local_total"] = data["local_total"].str.split(": ").str[-1]

In [25]:
data["local_total"] = (
    data["local_total"]
    .str.replace(",", "", regex=True)
    .str.replace("$", "", regex=True)
    .fillna(0)
    .astype("float")
)

In [26]:
monetary_cols = [
    "total_expenses",
    "_5311_funds",
    "_5311_f__funds",
    "_5311_cmaq_funds",
    "_5339_funds",
    "federal_total",
    "other_fed_funds_total",
    "lctop__state__funds",
    "sb1__state_of_good_repair__state__funds",
    "transit_development_act__state__funds",
    "other_state_funds",
    "state_total",
]

In [27]:
# Clean up monetary columns
data[monetary_cols] = (
    data[monetary_cols]
    .fillna(value=0)
    .apply(pd.to_numeric, errors="coerce")
    .astype("float")
)

In [28]:
data["total_state_federal_local_funding"] = (
    
    data["state_total"]
    + data["local_total"]
    + data["federal_total"]
    + data['other_state_funds']
    + data['other_fed_funds_total']
)

In [29]:
def funding_vs_expenses(df):
    if df["total_state_federal_local_funding"] == df["total_expenses"]:
        return "Fully funded"
    elif df["total_state_federal_local_funding"] > df["total_expenses"]:
        return "Funding exceeds total expenses"
    else:
        return "Not fully funded"

In [30]:
data["fully_funded"] = data.apply(funding_vs_expenses, axis=1)

In [31]:
data.fully_funded.value_counts()

Fully funded                      227
Funding exceeds total expenses     84
Not fully funded                   35
Name: fully_funded, dtype: int64

In [32]:
data[['project_upin', 'local_total']].sample(4)

,project_upin,local_total
62,BCG0003963,0.00
160,BCG0003948,190269.00
235,BCG0003823,37270.59
209,BCG0003924,0.00


## Melt dataframe
* Every project has a unique upin number, use that as the merge column.

In [33]:
len(data)

346

In [34]:
data.project_upin.nunique()

346

In [35]:
data.columns

Index(['organization_name', 'district', 'application_name', 'year',
       'application_status', 'project_upin', 'project_category',
       'project_line_item__ali_', 'project_description', 'is_stimulus',
       'consolidated_application', 'total_expenses', '_5311_funds',
       '_5311_f__funds', '_5311_cmaq_funds', '_5339_funds', 'federal_total',
       'other_fed_funds_total', 'lctop__state__funds',
       'sb1__state_of_good_repair__state__funds',
       'transit_development_act__state__funds', 'other_state_funds',
       'state_total', 'local_total', 'short_description',
       'total_state_federal_local_funding', 'fully_funded'],
      dtype='object')

In [36]:
# Keeping only monetary columns I want to melt
monetary_subset = data[
    [
        "project_upin",
        "_5311_funds",
        "_5311_f__funds",
        "_5311_cmaq_funds",
        "_5339_funds",
        "lctop__state__funds",
        "sb1__state_of_good_repair__state__funds",
        "transit_development_act__state__funds",
        "other_state_funds",
        "other_fed_funds_total",
        "local_total",
    ]
]

In [37]:
monetary_subset = pd.melt(
    monetary_subset,
    id_vars=["project_upin"],
    value_vars=[
        "_5311_funds",
        "_5311_f__funds",
        "_5311_cmaq_funds",
        "_5339_funds",
        "lctop__state__funds",
        "sb1__state_of_good_repair__state__funds",
        "transit_development_act__state__funds",
         "other_state_funds",
        "other_fed_funds_total",
        "local_total",
    ],
    var_name="program_name",
    value_name="funding_received",
)

In [38]:
# Delete some irrelevant cols from original data set
data2 = data[
    [
        "total_expenses",
        "organization_name",
        "district",
        "year",
        "application_status",
        "project_upin",
        "project_category",
        "project_line_item__ali_",
        "project_description",
        "is_stimulus",
        "total_state_federal_local_funding",
        "fully_funded",
        "short_description",
    ]
]

In [39]:
# merge original dataframe with melted dataframe to get full information.
merge1 = pd.merge(monetary_subset, data2, on="project_upin", how="left")

In [40]:
merge1.program_name.unique()

array(['_5311_funds', '_5311_f__funds', '_5311_cmaq_funds', '_5339_funds',
       'lctop__state__funds', 'sb1__state_of_good_repair__state__funds',
       'transit_development_act__state__funds', 'other_state_funds',
       'other_fed_funds_total', 'local_total'], dtype=object)

In [41]:
merge1["program_name"] = merge1["program_name"].replace(
    {
        '_5311_funds':'5311 (Fed)',
        'lctop__state__funds': 'LCTOP (State)',
        'transit_development_act__state__funds':'Transit Development Act (State)',
        'other_state_funds':'Other State Funds',
        '_5339_funds': '5339 (Fed)',
        '_5311_f__funds': '5311(f) (Fed)',
        'sb1__state_of_good_repair__state__funds': 'SB1. State of Good Repair (State)',
        'other_fed_funds_total': 'Other Federal Funds',
        '_5311_cmaq_funds': '5311 CMAQ (Fed)',
        'local_total': 'Local Funds'
        
    }
)

In [42]:
# looking at new length of merge...
len(merge1)

3460

In [43]:
# double checking that project upin count is still the same
merge1.project_upin.nunique()

346

In [44]:
merge1.program_name.unique()

array(['5311 (Fed)', '5311(f) (Fed)', '5311 CMAQ (Fed)', '5339 (Fed)',
       'LCTOP (State)', 'SB1. State of Good Repair (State)',
       'Transit Development Act (State)', 'Other State Funds',
       'Other Federal Funds', 'Local Funds'], dtype=object)

In [45]:
# filter any zeroes in the funding received column, to make dataframe smaller
melt_df = merge1[merge1["funding_received"] > 0]

In [46]:
melt_df.shape

(641, 15)

In [47]:
melt_df.sample()

,project_upin,program_name,funding_received,total_expenses,organization_name,district,year,application_status,project_category,project_line_item__ali_,project_description,is_stimulus,total_state_federal_local_funding,fully_funded,short_description
3331,BCG0003704,Local Funds,507807.00,1136796.00,North County Transit District,11.00,2022,Submitted,Operating,300902,operating assistance sliding scale - ffy22,No,1136796.00,Fully funded,Operating Assistance


### Why do upins  suddenly disappear??? 
* Investigation: look at  project upin #s in original dataframe versus pivoted.
* When I pivotted the data, I only included the granular categories (5311/sb1/etc), not the totals. I also filtered out for any $0.
* Found: Projects in "main_list" either
    * Has 0.00 in federal_total, local_total, and state_total
    * Somehow has 0.00 in 5311/5311f/5311cmaq/5339 funds but has the federal_total populated.
    

In [48]:

melt_df.project_upin.nunique()

339

In [49]:
#Investigate
melted_upin = melt_df.project_upin.unique().tolist()

In [50]:
len(melted_upin)

339

In [51]:
data_upin = data.project_upin.unique().tolist()

In [52]:
len(data_upin)

346

In [53]:
main_list = np.setdiff1d(data_upin,melted_upin)
main_list

array(['BCG0003702', 'BCG0003954', 'BCG0003959', 'BCG0004002',
       'BCG0004004', 'BCG0004030', 'BCG0004032'], dtype='<U10')

In [54]:
missing_upin = data[data["project_upin"].isin(['BCG0003702', 'BCG0003954', 'BCG0003959', 'BCG0004002',
       'BCG0004004', 'BCG0004030', 'BCG0004032'])]

In [55]:
# Exporting a dataframe to csv
#missing_upin.to_csv("./dataframe.csv", index=False)

## Program Groups

In [57]:
#filter out local funds
group = melt_df.loc[melt_df["program_name"] != "Local Funds"]


In [58]:
group.program_name.unique()

array(['5311 (Fed)', '5311(f) (Fed)', '5311 CMAQ (Fed)', '5339 (Fed)',
       'LCTOP (State)', 'SB1. State of Good Repair (State)',
       'Transit Development Act (State)', 'Other State Funds',
       'Other Federal Funds'], dtype=object)

In [59]:
#grab all the different program names by project upin and put it in a new column
group["all_programs"] = group.groupby("project_upin")["program_name"].transform(
    lambda x: ",".join(x)
)

/tmp/ipykernel_634/521858038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group["all_programs"] = group.groupby("project_upin")["program_name"].transform(


In [60]:
group = group[["project_upin", "all_programs"]].drop_duplicates()

In [61]:
group.shape

(335, 2)

In [62]:
group.sample(2)

,project_upin,all_programs
1426,BCG0003767,LCTOP (State)
303,BCG0003869,5311 (Fed)


In [63]:
count = Counter()
for row in group["all_programs"]:
    row_list = row.split(",")
    count.update(Counter(combinations(row_list, 3))) 

In [64]:
group_combos = pd.DataFrame.from_records(list(dict(count).items()), columns=['Combination','Count_of_projects_upin'])
group_combos

,Combination,Count_of_projects_upin
0,"(5311 (Fed), SB1. State of Good Repair (State), Transit Development Act (State))",7
1,"(5311 (Fed), LCTOP (State), SB1. State of Good Repair (State))",4
2,"(5311 (Fed), LCTOP (State), Transit Development Act (State))",6
3,"(5311 (Fed), LCTOP (State), Other State Funds)",6
4,"(5311 (Fed), SB1. State of Good Repair (State), Other State Funds)",4
5,"(5311 (Fed), Transit Development Act (State), Other State Funds)",6
6,"(LCTOP (State), SB1. State of Good Repair (State), Transit Development Act (State))",2
7,"(LCTOP (State), SB1. State of Good Repair (State), Other State Funds)",4
8,"(LCTOP (State), Transit Development Act (State), Other State Funds)",4
9,"(SB1. State of Good Repair (State), Transit Development Act (State), Other State Funds)",2


In [65]:
#merge for other information
grouped_df = pd.merge(group, data, on="project_upin", how="left")

In [66]:
#keep only relevant cols
grouped_df = grouped_df[["project_upin", "organization_name", "short_description","all_programs", "year"]]

In [67]:
#drop duplicates 
#grouped_df = grouped_df.drop_duplicates(subset=["organization_name", "all_programs", "year"])

In [68]:
#count number of items in all programs
# https://stackoverflow.com/questions/51502263/pandas-dataframe-object-has-no-attribute-str
grouped_df["count_of_funding_programs_applied"] = (
    grouped_df["all_programs"]
    .str.split(",+")
    .str.len()
    .groupby(grouped_df.project_upin)
    .transform("sum")
)

In [70]:
grouped_df.sort_values('count_of_funding_programs_applied', ascending = False).head(250)

,project_upin,organization_name,short_description,all_programs,year,count_of_funding_programs_applied
22,BCG0003953,City of Escalon,Operating Assistance,"5311 (Fed),LCTOP (State),SB1. State of Good Repair (State),Transit Development Act (State),Other State Funds",2022,5
125,BCG0003696,Town of Truckee,Operating Assistance,"5311 (Fed),LCTOP (State),Transit Development Act (State),Other State Funds,Other Federal Funds",2022,5
126,BCG0003971,Town of Truckee,Operating Assistance,"5311 (Fed),LCTOP (State),Transit Development Act (State),Other State Funds,Other Federal Funds",2022,5
20,BCG0003951,City of Escalon,Operating Assistance,"5311 (Fed),LCTOP (State),SB1. State of Good Repair (State),Transit Development Act (State),Other State Funds",2022,5
69,BCG0003977,Imperial County Transportation Commission,Operating Assistance,"5311 (Fed),Transit Development Act (State),Other State Funds,Other Federal Funds",2022,4
68,BCG0003910,Imperial County Transportation Commission,Operating Assistance,"5311 (Fed),Transit Development Act (State),Other State Funds,Other Federal Funds",2022,4
133,BCG0003857,Tuolumne County Transit Agency,Operating Assistance,"5311 (Fed),LCTOP (State),SB1. State of Good Repair (State),Other State Funds",2022,4
134,BCG0003858,Tuolumne County Transit Agency,Operating Assistance,"5311 (Fed),LCTOP (State),SB1. State of Good Repair (State),Other State Funds",2022,4
166,BCG0003818,Plumas County Transportation Commission,Operating Assistance,"5311(f) (Fed),SB1. State of Good Repair (State),Other State Funds",2022,3
132,BCG0003944,Tulare County Regional Transportation Agency,Operating Assistance,"5311 (Fed),Transit Development Act (State),Other Federal Funds",2022,3


## Export

In [71]:
with pd.ExcelWriter(f"{GCS_FILE_PATH}Con_App_Cleaned.xlsx") as writer:
    melt_df.to_excel(writer, sheet_name="pivoted_data", index=False)
    data.to_excel(writer, sheet_name="cleaned_unpivoted_data", index=False)
    grouped_df.to_excel(writer, sheet_name="combos_of_funding_programs", index=False)
